# Tugas 2

Prio Budi Laksono

210411100177

Preprocessing hasil crawling data dari tribunnews.com

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mendapatkan konten dari URL dengan User-Agent
def get_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Kesalahan saat mengambil {url}: {e}")
        return None

# Fungsi untuk mendapatkan detail artikel dari halaman detail
# Fungsi untuk mendapatkan detail artikel dari halaman detail dengan penambahan kategori dari variabel 'categories'
def get_article_details(detail_url, category_name):
    detail_soup = get_soup(detail_url)
    if detail_soup:
        content = ' '.join([p.text for p in detail_soup.find_all('p')])
        date_tag = detail_soup.find('time')
        date = date_tag.text.strip() if date_tag else 'Tidak ada tanggal'
        title_tag = detail_soup.find('h1')
        title = title_tag.text.strip() if title_tag else 'Tidak ada judul'

        # Gunakan kategori yang diambil dari variabel 'categories' alih-alih dari breadcrumb
        category = category_name

        return {
            'judul': title,
            'isi_berita': content,
            'tanggal': date,
            'kategori': category,
            'url': detail_url
        }
    return None


# Fungsi untuk mendapatkan artikel dari suatu kategori
def get_articles(category_url, category_name, max_articles=100):
    articles = []
    page = 1

    while len(articles) < max_articles:
        url = f'{category_url}?page={page}'
        print(f"Mengambil: {url}")
        soup = get_soup(url)

        if soup is None:
            break

        article_list = soup.find_all('h3')

        if not article_list:
            print(f"Tidak ada artikel ditemukan di halaman {page}.")
            break

        for article in article_list:
            if len(articles) >= max_articles:
                break

            title_tag = article.find('a')
            detail_url = title_tag['href'] if title_tag else None

            if detail_url:
                if not detail_url.startswith('http'):
                    detail_url = f'https://www.tribunnews.com{detail_url}'
                
                # Pemanggilan fungsi dengan kedua argumen
                article_details = get_article_details(detail_url, category_name)
                
                if article_details:
                    articles.append(article_details)

        page += 1
        time.sleep(2)  # Beri jeda agar tidak terlalu cepat melakukan permintaan

    return articles


# URL Kategori
categories = {
    'Otomotif': 'https://www.tribunnews.com/otomotif',
    'Sport': 'https://www.tribunnews.com/sport'
}

# Mengumpulkan data dari dua kategori
all_articles = []
for category_name, category_url in categories.items():
    print(f"Menambang kategori {category_name}...")
    articles = get_articles(category_url, category_name, max_articles=100)
    all_articles.extend(articles)

# Simpan ke dalam DataFrame
df = pd.DataFrame(all_articles)

# Simpan ke dalam file CSV
df.to_csv('tribunnews_articles.csv', index=False)

print("Data berhasil ditambang dan disimpan dalam 'tribunnews_articles.csv'.") 

Menambang kategori Otomotif...
Mengambil: https://www.tribunnews.com/otomotif?page=1
Mengambil: https://www.tribunnews.com/otomotif?page=2
Mengambil: https://www.tribunnews.com/otomotif?page=3
Menambang kategori Sport...
Mengambil: https://www.tribunnews.com/sport?page=1
Mengambil: https://www.tribunnews.com/sport?page=2
Mengambil: https://www.tribunnews.com/sport?page=3
Data berhasil ditambang dan disimpan dalam 'tribunnews_articles.csv'.


## Preprocessing Data

In [6]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Unduh stopwords jika belum diunduh
try:
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading stopwords: {e}")

# Fungsi untuk membersihkan dan melakukan stemming pada teks
def preprocess_text(text):
    # Hapus HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Hapus angka dan tanda baca
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Ubah menjadi huruf kecil
    text = text.lower()
    # Tokenisasi
    tokens = text.split()
    # Hapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

# Load data dari CSV
try:
    df = pd.read_csv(r'C:\Users\pblak\OneDrive\Desktop\Tugas Kuliah\Semester 7\Pencarian dan Penambangan WEB\tribunnews_articles.csv')
except FileNotFoundError:
    print("File tidak ditemukan. Pastikan path file CSV benar.")
    raise

# Terapkan preprocessing ke setiap konten artikel
df['isi_berita_clean'] = df['isi_berita'].apply(preprocess_text)

# Simpan ke CSV hasil preprocessing
df.to_csv(r'C:\Users\pblak\OneDrive\Desktop\Tugas Kuliah\Semester 7\Pencarian dan Penambangan WEB\tribunnews_articles_preprocessed.csv', index=False)

print("Preprocessing selesai dan hasil disimpan dalam 'tribunnews_articles_preprocessed.csv'.")


Preprocessing selesai dan hasil disimpan dalam 'tribunnews_articles_preprocessed.csv'.


## Membangun model logistic regression

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Membaca data yang telah diproses
df = pd.read_csv('tribunnews_articles_preprocessed.csv')

# Misalkan kita ingin mengklasifikasikan kategori berdasarkan 'isi_berita_clean'
X = df['isi_berita_clean']
y = df['kategori']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membangun pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Melatih model
pipeline.fit(X_train, y_train)

# Mengevaluasi model
accuracy = pipeline.score(X_test, y_test)
print(f"Akurasi model: {accuracy:.2f}")


Akurasi model: 0.90


## Berikut untuk streamlit lokalhostnya : http://localhost:8502/